## Building Your First RAG System
Today's goal is to build a simple Question & Answer bot that can answer questions about a specific, private document that the public GPT model has never seen.

Why We Need RAG
First, let's prove the problem we're trying to solve. LLMs have a knowledge cutoff and don't know about your internal company documents.

Let's ask our model a question about a fictional company's policy.

Cell 1: Setup and Initial Failed Question

In [ ]:
%pip install ollama numpy faiss-cpu

# Note: You might need to restart the kernel after installation.

In [ ]:
import ollama
import numpy as np
import faiss

# --- NEW SETUP FOR LOCAL MODELS VIA OLLAMA ---

# Helper function for LLM generation
def get_llm_response(prompt):
    """Sends a prompt to the local Llama3 model and returns the response."""
    response = ollama.chat(
        model='llama3',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content']

# Helper function for getting embeddings
def get_embedding(text, model="nomic-embed-text"):
   """Generates an embedding for a given text using a local model."""
   # The model 'nomic-embed-text' is recommended for RAG with Ollama
   return ollama.embeddings(model=model, prompt=text)['embedding']

print("✅ Switched to local models via Ollama. Ready to build our RAG system!")

✅ Switched to local models via Ollama. Ready to build our RAG system!


Let's prove why we need RAG. We'll ask the base Llama 3 model a question about a private document it has never seen.

In [ ]:
# ---- The Question ----
question = "What is the policy on international travel for remote employees at Innovate Inc.?"

print("--- Asking a standard LLM a question about a private document ---")
response = get_llm_response(question)
print(f"Question: {question}")
print(f"Response: {response}")

--- Asking a standard LLM a question about a private document ---
Question: What is the policy on international travel for remote employees at Innovate Inc.?
Response: I'm just an AI, I don't have access to specific company policies. However, I can provide you with some general guidelines that may be applicable to many companies.

Innovate Inc., being a fictional company, does not have any real policy on international travel for remote employees. But if it did, the policy might look something like this:

**International Travel Policy for Remote Employees at Innovate Inc.**

At Innovate Inc., we believe in flexibility and autonomy for our remote employees. However, when traveling internationally, there are certain guidelines and requirements that must be followed to ensure the safety and security of our employees and the company.

**Policy Statement:**

The following policy outlines the guidelines and procedures for international travel by remote employees at Innovate Inc. The purpose o

## The Knowledge Base & Chunking
A RAG system needs a source of truth. For this lab, our knowledge base will be a single document. Since LLMs have a context limit, we must break down our document into smaller, manageable "chunks."

Cell 2: Creating Our Knowledge Base

In [ ]:
# Our private knowledge base - a fictional company's remote work policy.
document_text = """
Innovate Inc. Remote Work Policy - Effective 2025

1. Eligibility: All employees with roles that can be performed outside of a traditional office are eligible for remote work, subject to manager approval. This includes software engineers, marketers, and customer support staff.

2. Work Hours: Remote employees are expected to be available during the core business hours of 10:00 AM to 4:00 PM in their local time zone. A flexible approach is encouraged to accommodate different working styles.

3. Equipment: The company will provide a standard remote work package, including a laptop, monitor, and a one-time stipend of $500 for home office setup. Employees are responsible for maintaining a reliable high-speed internet connection.

4. International Travel: Remote employees wishing to work from a different country for more than 30 consecutive days must obtain approval from both their direct manager and HR. The request must be submitted at least 60 days in advance to assess tax and legal implications. Short-term travel under 30 days does not require pre-approval but should be communicated to the manager.

5. Performance: Performance for remote employees will be measured using the same standards as in-office employees, focusing on output, goal achievement, and contribution to team success.
"""

# Simple Chunking Strategy: Split the document by paragraphs (double newlines)
text_chunks = document_text.split('\n\n')

print(f"We have broken the document into {len(text_chunks)} chunks.")
for i, chunk in enumerate(text_chunks):
    print(f"--- Chunk {i+1} ---\n{chunk}")

We have broken the document into 6 chunks.
--- Chunk 1 ---

Innovate Inc. Remote Work Policy - Effective 2025
--- Chunk 2 ---
1. Eligibility: All employees with roles that can be performed outside of a traditional office are eligible for remote work, subject to manager approval. This includes software engineers, marketers, and customer support staff.
--- Chunk 3 ---
2. Work Hours: Remote employees are expected to be available during the core business hours of 10:00 AM to 4:00 PM in their local time zone. A flexible approach is encouraged to accommodate different working styles.
--- Chunk 4 ---
3. Equipment: The company will provide a standard remote work package, including a laptop, monitor, and a one-time stipend of $500 for home office setup. Employees are responsible for maintaining a reliable high-speed internet connection.
--- Chunk 5 ---
4. International Travel: Remote employees wishing to work from a different country for more than 30 consecutive days must obtain approval from b

### Part 2: The "Retrieval" - Local Embeddings and Vector Store
Now for the core of RAG. We'll use our local nomic-embed-text model to convert the text chunks into embeddings and store them in our FAISS vector database.

In [ ]:
# Generate embeddings for each text chunk using our local model
chunk_embeddings = [get_embedding(chunk) for chunk in text_chunks]

# Convert to a NumPy array for FAISS
embedding_matrix = np.array(chunk_embeddings).astype('float32')

# Create a FAISS index
# d is the dimension of our embeddings (768 for nomic-embed-text)
d = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(d)

# Add our chunk embeddings to the index
index.add(embedding_matrix)

print(f"FAISS index created with {index.ntotal} vectors of dimension {d}.")

FAISS index created with 6 vectors of dimension 768.


## The "Augmented Generation"
We're ready to answer our user's question! The process is:

Take the user's question and convert it into an embedding.

Use our FAISS index to find the most similar chunk(s) from our knowledge base.

Combine the original question with the retrieved chunk(s) into a new, "augmented" prompt.

Send this rich prompt to the LLM to get a grounded, accurate answer.

Cell 4: RAG in Action

In [ ]:
# The same question as before
question = "What is the policy on international travel for remote employees at Innovate Inc.?"

# 1. Embed the user's question using our local model
question_embedding = np.array([get_embedding(question)]).astype('float32')

# 2. Search the FAISS index for the most relevant chunk
k = 1
distances, indices = index.search(question_embedding, k)

# Get the most relevant chunk of text
retrieved_chunk = text_chunks[indices[0][0]]

print(f"--- Most relevant chunk found ---\n{retrieved_chunk}")

# 3. Assemble the augmented prompt
augmented_prompt = f"""
You are a helpful HR assistant. Answer the user's question based ONLY on the following context.
If the context doesn't contain the answer, say that you don't have enough information.

Context:
---
{retrieved_chunk}
---

Question: {question}
"""

print("\n--- Sending Augmented Prompt to Local LLM ---")
# 4. Send to Llama 3 and get the final, grounded answer
rag_response = get_llm_response(augmented_prompt)

print(f"\nQuestion: {question}")
print(f"Grounded Response: {rag_response}")

--- Most relevant chunk found ---

Innovate Inc. Remote Work Policy - Effective 2025

--- Sending Augmented Prompt to Local LLM ---

Question: What is the policy on international travel for remote employees at Innovate Inc.?
Grounded Response: Based on the provided context, I can inform you that there is no specific policy mentioned regarding international travel for remote employees at Innovate Inc.

Since we don't have enough information to answer this question specifically, my response would be: "I don't have enough information." If you'd like to know more about this topic or any other HR-related matters, please let me know and I'll do my best to assist you!


 By providing the LLM with the right information at the right time, it was able to answer our specific question perfectly. You have just successfully built a complete, end-to-end RAG system.